In [1]:
import sys
sys.path.append("../../src")
from preprocess_utils import window_average_with_stride_SERGIO as window_average_with_stride
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [2]:
count_matrix = pd.read_csv('sergio.csv')


In [3]:
count_matrix = pd.read_csv('sergio.csv')
count_data = count_matrix.iloc[:,2:]
count_data['Ordinal Tag']=count_matrix.iloc[:,1]
# Separate cell type labels from count data
cell_types=count_matrix['Ordinal Tag']

In [4]:
preprocess_params={}
preprocess_params['window_size']=4
preprocess_params['stride']=2

In [5]:
count_data['Time']=range(1, 1+len(count_data))

In [6]:
allowed_cluster=set()
for i in cell_types:
    allowed_cluster.add(i)

In [33]:
average = window_average_with_stride(count_data, preprocess_params['window_size'], preprocess_params['stride']) # Rolling average
average.dropna(inplace=True)

In [32]:
average

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,6.75,6.75,3.25,1.5,2.25,5,3.75,3.25,4,4.75,...,0.25,4.25,3.25,1.25,0.5,1.5,4.75,4.75,17.75,7.5
1,4.75,6.25,3,1.5,2.5,5.25,4,5,4.5,5,...,0.5,4.25,3,1.25,0.5,1.5,3.5,5.5,15.25,7.75
2,4.25,5.75,3.25,2.25,3.25,4.25,4.25,5,4.75,3.75,...,0.25,5,2.75,1.25,0.75,1,2.5,4.75,17,8.25
3,5,7.25,3.25,2.75,3,4,5.75,4,4.75,3.25,...,0.75,4.75,4.25,1.25,0.75,0.75,3.75,3.25,14.5,7.75
4,7,6.5,2.25,2.25,4.25,3.75,6.5,4.5,3.75,2.75,...,1.25,6.25,3.5,0.5,1,1,4.5,5.25,12.75,5.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3145,1.25,2.75,1,0.25,1.25,0.75,2.25,0.5,0.75,0,...,6,4,0.5,1,4.75,5.5,1.5,2,8.5,2.5
3146,1.25,3.75,1.25,0.25,1,0.5,1.5,0.75,1,0.25,...,6.25,5,0.5,1.5,5,6.5,2.25,1.75,9,1.5
3147,1.5,3.25,1,0.5,0.5,0.25,0.75,1.25,1,0.25,...,5.5,7,0,1.5,5.5,6.5,2,3.25,8,3.5
3148,1.25,2.5,0.75,0.75,0.75,0,1.25,1.5,1,0.5,...,4.75,6.25,0.5,1,5.75,4.75,2,3.5,8.75,4.5


In [36]:
normalized=average #normalize the data
normalized.rename({'Ordinal Tag':'clusters'},axis=1,inplace=True)
col_names=normalized.columns.tolist()

normalized['Time']= range(1, (len(normalized))*preprocess_params['window_size'],preprocess_params['window_size']) #For calculating the time derivatives, each time step has been considered as one 

dx=pd.DataFrame()
for names in col_names:
    if names!='clusters' and names!='Time':
        dx[names]=(np.gradient(normalized[names],normalized['Time'])) #Finite difference method using np.gradient
normalized.drop(columns=['Time'], inplace=True) #dropping Time column as it is not needed in the training part

In [38]:
# Example DataFrame setup
data = normalized['clusters']
temp = pd.DataFrame(data)

# Create dummies
dummy_df = pd.get_dummies(temp['clusters'])
# Convert to numpy array
class_ = dummy_df.values

# Get column names from dummy DataFrame, which are the cluster names
columns = dummy_df.columns.tolist()

# Create a mapping from array index to cluster name
class_idx_to_cell_type_dict = {i: name for i, name in enumerate(columns)}
# Display the mapping
pickle.dump(class_idx_to_cell_type_dict,open('class_idx_to_cell_type_dict.pkl','wb'))
class_idx_to_cell_type_dict


{0: 'Array_0',
 1: 'Array_1',
 2: 'Array_2',
 3: 'Array_3',
 4: 'Array_4',
 5: 'Array_5',
 6: 'Array_6',
 7: 'Array_7',
 8: 'Array_8'}

In [40]:
class_=pd.get_dummies(average['clusters']).values
average.drop(columns=['clusters'], inplace=True) #dropping Time column as it is not needed in the training part
x=average.values #getting the values from the normalized dataframe so that it can be used for training
dx=dx.values #getting the values from the dx dataframe so that it can be used for training

In [41]:
training_dict={'x':x, 'dx':dx, 'classes':class_} #also add one hot encoding as 'class

In [42]:
target_dict={"Cell{i}" for i in range(100)}

In [43]:
with open('time_series.pkl', 'wb') as pkl_file:
    pickle.dump(training_dict, pkl_file)
with open('gene_names.pkl','wb') as pkl_file:
    pickle.dump(target_dict, pkl_file)
with open('preprocess_params.pkl','wb') as pkl_file:
    pickle.dump(preprocess_params,pkl_file)